In [4]:
fimport urllib3
import json
import requests
import io
import pandas as pd
import os
import numpy as np
import geopandas as gpd
from shapely.geometry import LineString
import matplotlib.pyplot as plt
# import contextily as ctx
# from matplotlib_scalebar.scalebar import ScaleBar

# functions

In [48]:
#dict
def get_all_dictionary_items(resource_name, q=None):
    """
    שליפת כל הרשומות ממילון מסוים ב-CBS API
    resource_name: שם המילון, לדוגמה 'localities', 'districts'
    מחזיר: pd.DataFrame
    """
    url = f"https://api.cbs.gov.il/Dictionary/geo/{resource_name}"
    all_items = []
    page = 1
    page_size = 250  # המקסימום שה-API מאפשר

    while True:
        params = {
            "expand": "false",
            "q":q,
            "format": "json",
            "download": "false",
            "page": page,
            "page_size": page_size
        }
        r = requests.get(url, params=params)
        data = r.json()

        # הוצאת הרשומות
        items = data.get("dictionary", {}).get("data", {}).get(resource_name, {}).get("items", [])
        if not items:
            break

        all_items.extend(items)

        # אם קיבלנו פחות מ-page_size → זה העמוד האחרון
        if len(items) < page_size:
            break

        page += 1

    # המרה ל-DataFrame שטוח
    df = pd.json_normalize(all_items)
    return df

# find the theme that interests me

# theme

In [9]:
def cbs_get(url, params=None):
    headers = {
        "User-Agent": "MyApp/1.0 (yodfatbenshalom@gmail.com)"  # חשוב מאוד
    }
    resp = requests.get(url, headers=headers, params=params)
    resp.raise_for_status()
    return resp.json()

In [182]:
# URL של הקטלוג הראשי
catalog_url = "https://apis.cbs.gov.il/series/catalog/level"
params = {"id":1,
          # 'subject':2,
          "lang": "he",
          "format": "json",
          "download": "false",

        }
r = requests.get(catalog_url, params=params)
# r.raise_for_status()
catalog = r.json()
df=pd.json_normalize(catalog['catalogs']['catalog'])
# הפיכת הקטלוג ל-DataFrame
df_catalog = pd.json_normalize(catalog)
#print(df_catalog.head())

In [183]:
df_catalog

,catalogs.catalog,catalogs.level,catalogs.paging.total_items,catalogs.paging.page_size,catalogs.paging.current_page,catalogs.paging.last_page,catalogs.paging.first_url,catalogs.paging.previous_url,catalogs.paging.current_url,catalogs.paging.next_url,catalogs.paging.last_url,catalogs.paging.base_url
0,"[{'path': [2, 1], 'name': 'אוכלוסייה, לפי קבוצ...",2,3,100,1,1,https://apis.cbs.gov.il//series/catalog/level?...,None,https://apis.cbs.gov.il//series/catalog/level?...,None,https://apis.cbs.gov.il//series/catalog/level?...,None


In [185]:
catalog_url = "https://apis.cbs.gov.il/series/catalog/path"
params = {"id":50,
          "lang": "he",
          "format": "json",
          "download": "false",

        }
r = requests.get(catalog_url, params=params)
# r.raise_for_status()
catalog = r.json()
df=pd.json_normalize(catalog['catalogs']['catalog'])
# הפיכת הקטלוג ל-DataFrame
#df_catalog = pd.json_normalize(catalog)
#print(df_catalog.head())
df

,path,name,pathDesc
0,"[50, 1, 1, 1, 7193]",שאתות ממאירות,None
1,"[50, 1, 1, 1, 7194]",סרטן השד,None
2,"[50, 1, 1, 1, 7195]",סרטן הערמונית,None
3,"[50, 1, 1, 1, 7196]",סרטן קנה הנשימה סמפונות וריאה,None
4,"[50, 1, 1, 1, 7197]",דמנציה,None
...,...,...,...
95,"[50, 1, 2, 3, 7356]",ערבים,None
96,"[50, 1, 2, 3, 7357]",חרדים,None
97,"[50, 1, 2, 3, 7358]",גילי 44-20,None
98,"[50, 1, 2, 3, 7359]",גילי 64-45,None


# get data

In [227]:
# 2. נניח שבחרת קוד נושא מהרשימה — עכשיו קבלת נתונים
data_url = "https://apis.cbs.gov.il/series/data/path"
data_params = {
    "id":"50,1,2,1,7357",  # דוגמה לסדרה
    "format": "json",
    "lang": "he",
    "addNull": "false",
    "data_hide": "false",
}
data = cbs_get(data_url, data_params)

In [228]:
# 3. יצירת DataFrame מהנתונים
if "DataSet" in data:
    df = pd.DataFrame(data["DataSet"]['Series'])
elif "data" in data:
    df = pd.DataFrame(data["data"])
else:
    # תסתכל בשדה אחר בהתאם למבנה JSON
    print("Structure:", data.keys())
    df = pd.DataFrame(data)

In [229]:
rows = []

for _, row in df.iterrows():
    series_id = row['id']
    time_unit = row['time']['name']
    data_type = row['data']['name']
    unit = row['unit']['name']
    calc = row['calc']['name']
    series_name = row['path']['level1']['name']  # אפשר להוסיף עוד רמות אם רוצים
    
    for obs in row['obs']:
        rows.append({
            'series_id': series_id,
            'series_name': series_name,
            'time_unit': time_unit,
            'data_type': data_type,
            'unit': unit,
            'calc': calc,
            'time': obs['TimePeriod'],
            'value': obs['Value']
        })

flat_df = pd.DataFrame(rows)
flat_df

,series_id,series_name,time_unit,data_type,unit,calc,time,value
0,122632,סטטיסטיקה מגדרית,שנה,נתונים מקוריים,אחוזים,,2022-01,94.3
1,122632,סטטיסטיקה מגדרית,שנה,נתונים מקוריים,אחוזים,,2021-01,96.9
2,122632,סטטיסטיקה מגדרית,שנה,נתונים מקוריים,אחוזים,,2020-01,94.4
3,122632,סטטיסטיקה מגדרית,שנה,נתונים מקוריים,אחוזים,,2019-01,94.0
4,122632,סטטיסטיקה מגדרית,שנה,נתונים מקוריים,אחוזים,,2018-01,95.1
5,122632,סטטיסטיקה מגדרית,שנה,נתונים מקוריים,אחוזים,,2017-01,93.8
6,122632,סטטיסטיקה מגדרית,שנה,נתונים מקוריים,אחוזים,,2016-01,95.6
7,122632,סטטיסטיקה מגדרית,שנה,נתונים מקוריים,אחוזים,,2015-01,94.4


In [226]:
data

{'DataSet': {'Series': [{'id': 122555,
    'ffu_num': None,
    'time': {'value': 1, 'name': 'שנה'},
    'data': {'value': 6, 'name': 'מתוקנני גיל'},
    'unit': {'value': 63, 'name': 'שיעור ל-100,000 תושבים'},
    'calc': {'value': 0, 'name': ''},
    'price': {'value': 0, 'name': ''},
    'path': {'level1': {'value': 50, 'name': 'סטטיסטיקה מגדרית'},
     'level2': {'value': 1, 'name': 'דמוגרפיה ובריאות'},
     'level3': {'value': 1, 'name': 'בריאות: תמותה'},
     'level4': {'value': 1, 'name': 'סיבות מוות, סך הכל'},
     'name_id': {'value': 7198, 'name': 'פגיעה עצמית מכוונת'}},
    'precis': 1,
    'update': '2023-12-17',
    'obs': [{'TimePeriod': '2021-01', 'Value': 3.8},
     {'TimePeriod': '2020-01', 'Value': 4.2},
     {'TimePeriod': '2019-01', 'Value': 4.3},
     {'TimePeriod': '2018-01', 'Value': 4.6},
     {'TimePeriod': '2017-01', 'Value': 4.2},
     {'TimePeriod': '2016-01', 'Value': 4.2},
     {'TimePeriod': '2015-01', 'Value': 3.8}]}],
  'paging': {'total_items': 1,
   '

# get dictionaries

In [193]:
# 2. נניח שבחרת קוד נושא מהרשימה — עכשיו קבלת נתונים
data_url = "https://apis.cbs.gov.il/series/data/list"
data_params = {
    "id":"7195",  # דוגמה לסדרה
    "format": "json",
    "lang": "he",
    "addNull": "false",
    "data_hide": "false"
}
data = cbs_get(data_url, data_params)

In [194]:
if "DataSet" in data:
    df = pd.DataFrame(data["DataSet"]['Series'])
elif "data" in data:
    df = pd.DataFrame(data["data"])
else:
    # תסתכל בשדה אחר בהתאם למבנה JSON
    print("Structure:", data.keys())
    df = pd.DataFrame(data)

rows = []

for _, row in df.iterrows():
    series_id = row['id']
    time_unit = row['time']['name']
    data_type = row['data']['name']
    unit = row['unit']['name']
    calc = row['calc']['name']
    series_name = row['path']['level1']['name']  # אפשר להוסיף עוד רמות אם רוצים

    for obs in row['obs']:
        rows.append({
            'series_id': series_id,
            'series_name': series_name,
            'time_unit': time_unit,
            'data_type': data_type,
            'unit': unit,
            'calc': calc,
            'time': obs['TimePeriod'],
            'value': obs['Value']
        })

flat_df = pd.DataFrame(rows)
flat_df

,series_id,series_name,time_unit,data_type,unit,calc,time,value
0,7195,סחר חוץ,חודש,נתונים מקוריים (ב $US ),מיליוני דולרים,,2025-10,0.000
1,7195,סחר חוץ,חודש,נתונים מקוריים (ב $US ),מיליוני דולרים,,2025-09,0.000
2,7195,סחר חוץ,חודש,נתונים מקוריים (ב $US ),מיליוני דולרים,,2025-08,0.002
3,7195,סחר חוץ,חודש,נתונים מקוריים (ב $US ),מיליוני דולרים,,2025-07,0.000
4,7195,סחר חוץ,חודש,נתונים מקוריים (ב $US ),מיליוני דולרים,,2025-06,0.123
...,...,...,...,...,...,...,...,...
95,7195,סחר חוץ,חודש,נתונים מקוריים (ב $US ),מיליוני דולרים,,2017-11,0.000
96,7195,סחר חוץ,חודש,נתונים מקוריים (ב $US ),מיליוני דולרים,,2017-10,0.000
97,7195,סחר חוץ,חודש,נתונים מקוריים (ב $US ),מיליוני דולרים,,2017-09,0.018
98,7195,סחר חוץ,חודש,נתונים מקוריים (ב $US ),מיליוני דולרים,,2017-08,0.005


In [167]:
df

,id,ffu_num,time,data,unit,calc,price,path,precis,update,obs
0,963,None,"{'value': 3, 'name': 'חודש'}","{'value': 1, 'name': 'נתונים מקוריים'}","{'value': 20, 'name': 'מספר'}","{'value': 0, 'name': ''}","{'value': 0, 'name': ''}","{'level1': {'value': 17, 'name': 'תחבורה ותקשו...",0,2015-03-04,"[{'TimePeriod': '2010-12', 'Value': 64.0}, {'T..."


d# query

In [59]:
# דוגמה לשימוש:
#[districts, localities, population_groups, municipal_or_council_status, metropolins,natural_areas,organizational_affiliations,
# planning_committee,regions, religions, locality_types,  localities]
# q='JERUSALEM'
dicta = get_all_dictionary_items("localities", q='JERUSALEM')
dicta

""
0


In [60]:
dicta

""
0


# time series

In [19]:
def get_cbs_timeseries(series_id, start_date=None, end_date=None):
    """
    פונקציה למשיכת סדרת זמן מה-API של הלמ״ס כולל מילון.

    Parameters:
    - series_id (str): קוד הסדרה
    - start_date (str, optional): תאריך התחלה בפורמט 'YYYY-MM-DD'
    - end_date (str, optional): תאריך סיום בפורמט 'YYYY-MM-DD'

    Returns:
    - pd.DataFrame: DataFrame עם הנתונים
    """
    url_data = f"https://api.cbs.gov.il/series/{series_id}/data"
    url_metadata = f"https://api.cbs.gov.il/series/{series_id}/metadata"

    # פרמטרים לשליפת נתונים
    params = {}
    if start_date:
        params['start_date'] = start_date
    if end_date:
        params['end_date'] = end_date

    # שליפת נתונים
    r_data = requests.get(url_data, params=params)
    r_metadata = requests.get(url_metadata)

    if r_data.status_code != 200:
        raise Exception(f"Error fetching data: {r_data.status_code}")
    if r_metadata.status_code != 200:
        raise Exception(f"Error fetching metadata: {r_metadata.status_code}")

    data_json = r_data.json()
    metadata_json = r_metadata.json()

    # המרה ל-DataFrame
    df_data = pd.json_normalize(data_json.get('values', []))

    # הוספת מידע מהמילון
    df_data['series_name'] = metadata_json.get('name')
    df_data['unit'] = metadata_json.get('unit')
    df_data['description'] = metadata_json.get('description')

    return df_data

# דוגמה לשימוש בפונקציה:
series_id = "234001"  # החלף בקוד סדרה אמיתי
df = get_cbs_timeseries(series_id, start_date="2020-01-01", end_date="2023-12-31")

print(df.head())
print(f"סה\"כ שורות: {len(df)}")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [20]:
url = "https://api.cbs.gov.il/series/catalog"
r = requests.get(url)
catalog = r.json()

# הצגת 5 סדרות לדוגמה
for series in catalog[:5]:
    print(series['id'], series['name'], series.get('description'))

JSONDecodeError: Expecting value: line 1 column 1 (char 0)